In [1]:
import torch

In [11]:
# Тензор с нормальным распределением (среднее 0, стандартное отклонение 1)
normal_tensor = torch.randn((3, 3))

# Тензор с равномерным распределением от 0 до 1
uniform_tensor = torch.rand((3, 3))

In [12]:
print(normal_tensor)
print(uniform_tensor)

tensor([[-0.2229, -1.1428, -0.0171],
        [ 0.2290, -0.5734, -2.2093],
        [-1.9888, -0.8727, -1.7653]])
tensor([[0.2016, 0.8269, 0.1905],
        [0.5261, 0.1697, 0.9757],
        [0.0796, 0.3943, 0.0133]])


In [13]:
normal_tensor + uniform_tensor

tensor([[-0.0213, -0.3159,  0.1734],
        [ 0.7551, -0.4036, -1.2335],
        [-1.9092, -0.4784, -1.7519]])

In [14]:
normal_tensor * uniform_tensor

tensor([[-0.0449, -0.9449, -0.0032],
        [ 0.1205, -0.0973, -2.1556],
        [-0.1583, -0.3441, -0.0236]])

In [15]:
normal_tensor.numpy()

array([[-0.22287978, -1.1427733 , -0.01705786],
       [ 0.22903423, -0.5733866 , -2.209275  ],
       [-1.9888078 , -0.8727395 , -1.7652754 ]], dtype=float32)

# 2 

In [21]:
# Создаем тензор размера 4x4, заполненный случайными числами
tensor = torch.rand((4, 4))

print("Original Tensor:")
print(tensor)

# Получаем центральный 2x2 блок срезами
central_block = tensor[1:3, 1:3]

# Изменяем значения элементов в центральном блоке на 1
central_block.fill_(1)

print("\nModified Tensor:")
print(tensor)

Original Tensor:
tensor([[0.0785, 0.6435, 0.8386, 0.9792],
        [0.8248, 0.5619, 0.0903, 0.3506],
        [0.1622, 0.1290, 0.7289, 0.5927],
        [0.4801, 0.9543, 0.4592, 0.6445]])

Modified Tensor:
tensor([[0.0785, 0.6435, 0.8386, 0.9792],
        [0.8248, 1.0000, 1.0000, 0.3506],
        [0.1622, 1.0000, 1.0000, 0.5927],
        [0.4801, 0.9543, 0.4592, 0.6445]])


# 3

In [24]:
# Создаем тензор x с requires_grad=True
x = torch.tensor(3.0, requires_grad=True)

# Определяем функцию y = 5x^3 - 2x^2 + x + 1
y = 5 * x**3 - 2 * x**2 + x + 1

# Вычисляем градиент функции y в точке x = 3.0
y.backward()

# Градиент хранится в x.grad
print("Gradient of y at x = 3.0:", x.grad)

Gradient of y at x = 3.0: tensor(124.)


# 4

In [42]:
import torch.nn as nn

class SimpleNN(nn.Module):
    def __init__(self, input_size, output_size):
        super(SimpleNN, self).__init__()
        self.layer1 = nn.Linear(input_size, 5)  # Первый полносвязный слой (10 -> 5)
        self.relu = nn.ReLU()                   # Функция активации ReLU
        self.layer2 = nn.Linear(5, output_size) # Второй полносвязный слой (5 -> 2)

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        return x

# Создаем модель
model = SimpleNN(input_size=10, output_size=2)

# Проверяем размер выходного тензора
input_tensor = torch.rand((1, 10))  # Случайный вход размером (1, 10)
output_tensor = model(input_tensor)

print("Output Tensor Size:", output_tensor.size())

Output Tensor Size: torch.Size([1, 2])


# 5

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Проверяем, доступно ли устройство MPS
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    print("MPS device found and will be used.")
else:
    print("MPS device not found.")
    mps_device = torch.device("cpu") 

# Задаем трансформации для нормализации данных MNIST
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Загружаем тренировочные и тестовые данные MNIST
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Создаем DataLoader для батчей
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Определяем модель нейронной сети с двумя линейными слоями
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.layer1 = nn.Linear(28*28, 128)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        return x

# Инициализируем модель, функцию потерь и оптимизатор
model = SimpleNN().to(mps_device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Обучение модели
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        # Перемещаем данные на нужное устройство
        inputs, labels = inputs.to(mps_device), labels.to(mps_device)

        # Обнуляем градиенты
        optimizer.zero_grad()

        # Прямой проход
        outputs = model(inputs)

        # Вычисляем потерю
        loss = criterion(outputs, labels)

        # Обратный проход и обновление весов
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")

# Оценка точности на тестовом наборе данных
correct = 0
total = 0
with torch.no_grad():  # Не вычисляем градиенты во время тестирования
    for inputs, labels in test_loader:
        # Перемещаем данные на нужное устройство
        inputs, labels = inputs.to(mps_device), labels.to(mps_device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on test dataset: {accuracy}%")


MPS device found and will be used.
Epoch [1/5], Loss: 0.46391023040564455
Epoch [2/5], Loss: 0.22583259104776865
Epoch [3/5], Loss: 0.17078456466894415
Epoch [4/5], Loss: 0.13758018934157992
Epoch [5/5], Loss: 0.11578888753985069
Accuracy on test dataset: 96.78%


# 6

In [41]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Проверяем, доступно ли устройство MPS
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    print("MPS device found and will be used.")
else:
    print("MPS device not found.")
    mps_device = torch.device("cpu") 

# Задаем трансформации для нормализации данных MNIST
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Загружаем тренировочные и тестовые данные MNIST
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Создаем DataLoader для батчей
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Определяем модель нейронной сети с двумя линейными слоями
class DropoutNN(nn.Module):
    def __init__(self):
        super(DropoutNN, self).__init__()

        self.layers = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.Dropout(),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(128, 10),
        )

    def forward(self, x):
        x = x.view(-1, 28*28)
        return self.layers(x)

# Инициализируем модель, функцию потерь и оптимизатор
model = DropoutNN().to(mps_device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Обучение модели
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        # Перемещаем данные на нужное устройство
        inputs, labels = inputs.to(mps_device), labels.to(mps_device)

        # Обнуляем градиенты
        optimizer.zero_grad()

        # Прямой проход
        outputs = model(inputs)

        # Вычисляем потерю
        loss = criterion(outputs, labels)

        # Обратный проход и обновление весов
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")

# Оценка точности на тестовом наборе данных
correct = 0
total = 0
with torch.no_grad():  # Не вычисляем градиенты во время тестирования
    for inputs, labels in test_loader:
        # Перемещаем данные на нужное устройство
        inputs, labels = inputs.to(mps_device), labels.to(mps_device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on test dataset: {accuracy}%")


MPS device found and will be used.
Epoch [1/5], Loss: 0.742285644861935
Epoch [2/5], Loss: 0.4529536496093278
Epoch [3/5], Loss: 0.4011625550798516
Epoch [4/5], Loss: 0.37418753988167114
Epoch [5/5], Loss: 0.3518090529212438
Accuracy on test dataset: 89.81%


# 7